In [2]:
import os
import warnings
import shutil
import zipfile
import random
import numpy as np
import glob
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from distutils.version import LooseVersion
from urllib.request import urlretrieve
from tqdm import tqdm
from PIL import Image
import scipy.misc


In [7]:
class FCN_8():
    def __init__(self,data_path='./data/',image_shape=(160,576),num_classes=2,epochs=10,batch_size=20,learning_rate=0.0005,save_model=True):
        
        self.GPU=self.check_GPU()
        self.data_path=data_path
        self.TF_version=self.check_version()
        self.dataset_folders=self.downlaod_dataset(self.data_path)
        self.vgg_weights_path=self.downlaod_pretrained_vgg(self.data_path)
        self.FCN_weights_path=os.path.join(self.data_path,'fcn_model/FCN.ckpt')      
        self.num_classes=num_classes
        self.image_shape=image_shape
        self.epochs=epochs
        self.batch_size=batch_size
        self.save_model=save_model
        self.learning_rate=learning_rate

        
        
    def check_version(self):
        assert LooseVersion(tf.__version__)>=LooseVersion('1.0'),'Please use Tensorflow version 1.0 or newer. You are using {}'.format(tf.__version__)
        return tf.__version__
        
    def check_GPU(self):
        if not tf.test.gpu_device_name():
            warnings.warn('Non GPU found. Please use a GPU to train your neural network.')
            return False
        else:
            print('Default GPU Devices{}'.format(tf.test.gpu_device_name()))
            return True
        
    def downlaod_pretrained_vgg(self,path):
        """
        Download and extract pretrained vgg model if doesnt exist
        :param path: Directory to downlaod the model to
        """
        vgg_file_name="vgg.zip"
        vgg_path=os.path.join(path,'vgg_weights')
        vgg_files=[os.path.join(vgg_path,'vgg/variables/variables.data-00000-of-00001'),
                  os.path.join(vgg_path,'vgg/variables/variables.index'),
                  os.path.join(vgg_path,'vgg/saved_model.pb')]
        missing_files=[vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
        #print(missing_files)
        if missing_files:
            #Clean vgg directory
            if os.path.exists(vgg_path):
                #clean vgg path
                shutil.rmtree(vgg_path)
            os.mkdir(vgg_path)
            
            print('Downloading pretrained vgg nodel...')
            with DLProgress(unit='B',unit_scale=True,miniters=1) as pbar:
                urlretrieve('https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
                           os.path.join(path,vgg_file_name),
                           pbar.hook)
        
            #Extract vgg file
            zip_ref=zipfile.ZipFile(os.path.join(path,vgg_file_name),'r')
            zip_ref.extractall(vgg_path)
            zip_ref.close()
            os.remove(os.path.join(path, vgg_filename))
        else:
            print('Vgg weights are available')
        return os.path.join(vgg_path,'vgg')         
                                
                                
    def load_vgg(self,sess):
        """
        Load Pretrained VGG Model into TensorFlow
        :param sess: TensorFlow Session
        :param vgg_weights_path: the path to the vgg folder where contains weights variables
        :return: Tuple of Tensors from VGG Model(Input Image,Keep_probab,layer3_out,layer4_out,layer7_out)
        """
        vgg_name='vgg16'
        vgg_input_tensor_name='image_input:0'
        vgg_keep_prob_tensor_name='keep_prob:0'
        vgg_layer3_out_tensor_name='layer3_out:0'
        vgg_layer4_out_tensor_name='layer4_out:0'
        vgg_layer7_out_tensor_name='layer7_out:0'
        
        tf.saved_model.loader.load(sess,[vgg_name],self.vgg_weights_path)
        graph=tf.get_default_graph()
        image_input=graph.get_tensor_by_name(vgg_input_tensor_name)
        keep_prob=graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
        layer3_out=graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
        layer4_out=graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
        layer7_out=graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

        return image_input,keep_prob,layer3_out,layer4_out,layer7_out
    
    def encoder(self,layer7_out,num_classes):
        """
        Create the encoder portion of the FCN 
        :param layer7_out:TF Tensor for VGG Layer 7 output
        :return: tensor for the last layer of the encoder which ic 1 by 1 convolution 
        """
        #1*1 conv
        conv11_out=tf.layers.conv2d(layer7_out,num_classes,1,
                                padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),name='conv11_out')
        return conv11_out
    
    def decoder(self,conv11_out,layer3_out,layer4_out,num_classes):
        
        """
        Create the Eecoder portion of the FCN 
        :param conv11_out:TF Tensor for last layer of the encoder 
        :param layer3_out:TF Tensor for VGG Layer 3 output
        :param layer4_out:TF Tensor for VGG Layer 4 output
        :return: tensor for the last layer of the decoder.
        """
        
        #upsample
        l8_out=tf.layers.conv2d_transpose(conv11_out,num_classes,4,2,
                                         padding='same',
                                         kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        # check the shapes
        l4=tf.layers.conv2d(layer4_out,num_classes,1,padding='same',
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                             kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        # skip connection (element-wise addition)
        l9_in=tf.add(l8_out,l4)
        
        #upsample by 2
        l9_out=tf.layers.conv2d_transpose(l9_in,num_classes,4,2,
                                         padding='same',
                                         kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        
        l3=tf.layers.conv2d(layer3_out,num_classes,1,padding='same',
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                             kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        # skip connection (element-wise addition)
        l10_in=tf.add(l9_out,l3)
        
        #upsample
        ll0_out = tf.layers.conv2d_transpose(l10_in, num_classes, 16, 8,
                                    padding='same',
                                    kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        
        return ll0_out
    
    def downlaod_dataset(self,path):
        """
        To load data from the the data directory
        :param data_folder: path to the folder contains all the dataset
        :return :lists of images and ground truths path
        """
        images_path=[]
        gt_images_path=[]


        dataset_file_name="data_road.zip"
        dataset_path=os.path.join(path,'data_road')
        dataset_folders=[os.path.join(path,'data_road/data_road/training/image_2/'),
                  os.path.join(path,'data_road/data_road/training/gt_image_2/'),
                  os.path.join(path,'data_road/data_road/testing/image_2/')]
        
        missing_folder=[dataset_folder for dataset_folder in dataset_folders if not os.path.exists(dataset_folder)]
        #print(missing_folder)
        if missing_folder:
            #Clean vgg directory
            if os.path.exists(dataset_path):
                #clean vgg path
                shutil.rmtree(dataset_path)
            os.mkdir(dataset_path)
            
            print('Downloading dataset...')
            with DLProgress(unit='B',unit_scale=True,miniters=1) as pbar:
                urlretrieve('https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip',
                           os.path.join(path,dataset_file_name),
                           pbar.hook)
        
            #Extract vgg file
            zip_ref=zipfile.ZipFile(os.path.join(path,dataset_file_name),'r')
            zip_ref.extractall(dataset_path)
            zip_ref.close()
            os.remove(os.path.join(path, dataset_file_name))
        else:
            print('Dataset is available')
            
          
        #gt_images_path={re.sub(r'_(road|lane)_','_',os.path.basename(path)):path for path in }
        return dataset_folders
    
    def generator(self,batch_size,image_shape):
        """
        create batches of training data
        :param data_folder: Path to folder that contains all the datasets
        :param image_shape: Tuple - Shape of image
        :param batch_size: Batch Size
        :return: Batches of training data
        """
        train_imgs=glob.glob(self.dataset_folders[0]+ '*.png')
        train_gts=glob.glob(self.dataset_folders[1]+ '*_road_*.png')
        train_imgs=sorted(train_imgs)
        train_gts=sorted(train_gts)
        
        background_color = np.array([255, 0, 0])
        
        #Shuffle two list at once with same order
        c=list(zip(train_imgs,train_gts))
        random.shuffle(c)
        train_imgs,train_gts=zip(*c)
        
        for batch in range(0,len(train_imgs),batch_size):
            X_train=[]
            y_train=[]
            
            train_batches=train_imgs[batch:batch+batch_size]
            train_Gtbatches=train_gts[batch:batch+batch_size]
            
            for image_file,gt_image_file in zip(train_batches,train_Gtbatches):
                
                image = cv2.resize(cv2.cvtColor(cv2.imread(image_file),cv2.COLOR_BGR2RGB), image_shape)
                gt_image =cv2.resize(cv2.cvtColor(cv2.imread(gt_image_file),cv2.COLOR_BGR2RGB), image_shape)

                gt_bg = np.all(gt_image == background_color, axis=2)

                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)
                
                X_train.append(image)
                y_train.append(gt_image)

            yield np.array(X_train), np.array(y_train)
                

    def optimize(self,nn_last_layer,correct_label,learning_rate,num_classes):
        
        """
        Build the TensorFLow loss and optimizer operations.
        :param nn_last_layer: TF Tensor of the last layer in the neural network
        :param correct_label: TF Placeholder for the correct label image
        :param learning_rate: TF Placeholder for the learning rate
        :param num_classes: Number of classes to classify
        :return: Tuple of (logits, train_op, cross_entropy_loss)
        """
        logits=tf.reshape(nn_last_layer,(-1,num_classes),name='logits')
        fix_label=tf.reshape(correct_label,(-1,num_classes))
        cross_entropy_loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=fix_label,logits=logits))
        optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op=optimizer.minimize(cross_entropy_loss)
        return logits,train_op,cross_entropy_loss
    
        
    def train_nn(self,sess,epochs,batch_size,train_op,cross_entropy_loss,
                    image_input,image_shape,correct_label,keep_prob,learning_rate):
         
        """
        Train neural network and print out the loss during training.
        :param sess: TF Session
        :param epochs: Number of epochs
        :param batch_size: Batch size
        :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
        :param train_op: TF Operation to train the neural network
        :param cross_entropy_loss: TF Tensor for the amount of loss
        :param input_image: TF Placeholder for input images
        :param correct_label: TF Placeholder for label images
        :param keep_prob: TF Placeholder for dropout keep probability
        :param learning_rate: TF Placeholder for learning rate
        """
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        if not os.path.exists(os.path.dirname(self.FCN_weights_path)):
            os.mkdir(os.path.dirname(self.FCN_weights_path))
        
        for i in tqdm(range(epochs)):
            print("Epoch >>>{}...".format(i+1))
            for image,label in self.generator(batch_size,image_shape):
                
                _,loss=sess.run([train_op,cross_entropy_loss],feed_dict={image_input:image,correct_label:label,keep_prob: 0.5,
                                          learning_rate: self.learning_rate})
                print("LOSS >>>> {:.3f}".format(loss))
                if self.save_model:
                     saver.save(sess, self.FCN_weights_path)
                
        print('Training Finished. Saved the weights to: {}'.format(self.FCN_weights_path))  
        
    def gen_test_output(self,sess, logits, keep_prob, image_pl, image_shape):
        """
        Generate test output using the test images and save them
        :param sess: TF session
        :param logits: TF Tensor for the logits
        :param keep_prob: TF Placeholder for the dropout keep robability
        :param image_pl: TF Placeholder for the image placeholder
        :param image_shape: Tuple - Shape of image
        """
        imgs=glob.glob(self.dataset_folders[2]+ '*.png')
        
        for img in imgs:
            image = cv2.resize(cv2.cvtColor(cv2.imread(img),cv2.COLOR_BGR2RGB), image_shape)

            im_softmax = sess.run([tf.nn.softmax(logits)],{keep_prob: 1.0, image_pl: [image]})


            im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
            segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
            mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
            #mask = scipy.misc.toimage(mask, mode="RGBA")
            mask = Image.fromarray(mask, mode="RGBA")
            street_im = Image.fromarray(image)
            street_im.paste(mask, box=None, mask=mask)
            if not os.path.exists(os.path.dirname(img)[:35]+'result/'):
                os.mkdir(os.path.dirname(img)[:35]+'result/')
            output_dir=os.path.join(os.path.dirname(img)[:35],'result/')+img.split('\\')[-1]
            plt.imsave(output_dir, image)

                
    def run_train(self):
        
        with tf.Session() as sess:
            
            correct_label=tf.placeholder(tf.int32,[None,None,None,self.num_classes],name='correct_label')
            learning_rate=tf.placeholder(tf.float32,name='learning_rate')
            image_input,keep_prob,layer3_out,layer4_out,layer7_out=self.load_vgg(sess)
            encoder=self.encoder(layer7_out,self.num_classes)
            decoder_last_layer=self.decoder(encoder,layer3_out,layer4_out,self.num_classes)
            logits,train_op,cross_entropy_loss=self.optimize(decoder_last_layer,correct_label,learning_rate,self.num_classes)
            self.train_nn(sess,self.epochs,self.batch_size,train_op,cross_entropy_loss,
                          image_input,self.image_shape,correct_label,keep_prob,learning_rate)  
    
    def run_test(self):
        
        with tf.Session() as sess:
                        
            correct_label=tf.placeholder(tf.int32,[None,None,None,self.num_classes],name='correct_label')
            learning_rate=tf.placeholder(tf.float32,name='learning_rate')
            image_input,keep_prob,layer3_out,layer4_out,layer7_out=self.load_vgg(sess)
            encoder=self.encoder(layer7_out,self.num_classes)
            decoder_last_layer=self.decoder(encoder,layer3_out,layer4_out,self.num_classes)
            logits,train_op,cross_entropy_loss=self.optimize(decoder_last_layer,correct_label,learning_rate,self.num_classes)
            saver = tf.train.Saver()
            saver.restore(sess, self.FCN_weights_path)
            print("Model restored.")
            self.gen_test_output(sess, logits, keep_prob, image_input, self.image_shape)    

class DLProgress(tqdm):
    last_bloack=0

    def hook(self,block_num=1,block_size=1,total_size=None):
        self.total=total_size
        self.update((block_num-self.last_bloack)*block_size)
        self.last_bloack=block_num   
        
    

In [9]:
fcn=FCN_8(epochs=3,batch_size=1,image_shape=(375, 1242),learning_rate=0.0005,save_model=True)
#print(fcn.dataset_folders)
#print(fcn.weights_path)

Dataset is available
Vgg weights are available


C:\Users\ziaeeamir\AppData\Local\Continuum\anaconda3\envs\Huber\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Non GPU found. Please use a GPU to train your neural network.


In [ ]:
fcn.run_train()

In [ ]:
fcn.run_test()